In [1]:
import numpy as np
import requests
import time

from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from distkeras.trainers import *
from distkeras.predictors import *
from distkeras.transformers import *
from distkeras.evaluators import *
from distkeras.utils import *

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
# Load the dataset and labels
x=np.load('NBx.npy')
y=np.load('NBy.npy')

In [3]:
conf = SparkConf()
sc = SparkCont
ext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0], n_init=10)
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"])
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"])
        model = Sequential()
        model.add(Dense(128,input_dim=fd.shape[1],activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(64,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(32,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(1,activation='sigmoid',use_bias=True)) #The number of neurons is equal to the number of classes
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        trainer = SingleTrainer(keras_model=model, worker_optimizer='adam', loss='binary_crossentropy', num_epoch=1000)
        trained_model = trainer.train(TrD)
        st = time.time()
        predictor = ModelPredictor(keras_model=trained_model)
        et=time.time()-st
        ff=predictor.predict(TsD)
        ts=np.array(map(lambda x: x[0],ff.select('prediction').collect())).reshape(-1,1)
        pp=pp+metrics.accuracy_score(y[test].reshape(-1,1),(ts>=0.5).astype(int))
        lpa=np.vstack((lpa,np.hstack((y[test].reshape(-1,1),ts))))
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time required to predict a label per each object in uS.

In [8]:
smr

[[78.0, 0.9771, 0.0531],
 [77.0, 0.9747, 0.0546],
 [76.0, 0.9752, 0.0584],
 [75.0, 0.9748, 0.0544],
 [74.0, 0.9755, 0.0532],
 [73.0, 0.9737, 0.0559],
 [72.0, 0.9738, 0.0548],
 [71.0, 0.9744, 0.05],
 [70.0, 0.9766, 0.0537],
 [69.0, 0.9755, 0.0545],
 [68.0, 0.977, 0.0548],
 [67.0, 0.9758, 0.0575],
 [66.0, 0.9768, 0.0571],
 [65.0, 0.9747, 0.0558],
 [64.0, 0.9736, 0.0558],
 [63.0, 0.9745, 0.0519],
 [62.0, 0.9742, 0.0552],
 [61.0, 0.9769, 0.0539],
 [60.0, 0.9743, 0.0541],
 [59.0, 0.9772, 0.0528],
 [58.0, 0.9723, 0.0594],
 [57.0, 0.9719, 0.0662],
 [56.0, 0.9715, 0.07],
 [55.0, 0.9726, 0.0613],
 [54.0, 0.9721, 0.0646],
 [53.0, 0.9736, 0.0687],
 [52.0, 0.9722, 0.065],
 [51.0, 0.9701, 0.0703],
 [50.0, 0.9714, 0.0676],
 [49.0, 0.9719, 0.0687],
 [48.0, 0.9707, 0.0645],
 [47.0, 0.9724, 0.0596],
 [46.0, 0.9719, 0.0677],
 [45.0, 0.972, 0.0615],
 [44.0, 0.9704, 0.0712],
 [43.0, 0.9717, 0.0646],
 [42.0, 0.9715, 0.0689],
 [41.0, 0.9721, 0.0636],
 [40.0, 0.9744, 0.0806],
 [39.0, 0.975, 0.0786],
 [38.0, 